In [89]:
import pandas as pd
import os

excel_file = "Preguntas.xlsx"
path = os.path.abspath(excel_file)
df = pd.read_excel(path)


In [90]:
def get_correct_answers():
    return df["Preguntas"][df["Respuesta"].notnull()].to_list()

def get_questions_and_options():
    data = df["Preguntas"].to_list()
    num_of_rows = df.shape[0]
    questions_and_options = dict()
    for i in range(0, num_of_rows, 5):
        questions_and_options[data[i]] = [
            data[i + 1], data[i + 2], data[i + 3], data[i + 4]
        ]
    return questions_and_options

In [91]:
from apiclient import discovery # API de cliente.
from httplib2 import Http # Necesaria para realizar las consultas a la API de Google.
from oauth2client.service_account import (
    ServiceAccountCredentials
) # Protocolo de seguridad.

SCOPES = "https://www.googleapis.com/auth/forms.body"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"
creds = ServiceAccountCredentials.from_json_keyfile_name(
    'key/credentials.json',
    SCOPES
)
http = creds.authorize(Http())

form_service = discovery.build(
    'forms',
    'v1',
    http=http,
    discoveryServiceUrl=DISCOVERY_DOC,
    static_discovery=False
)

NEW_FORM = {
    "info": {
        "title": "Guía de estudio para SAFe",
    }
}

UPDATE = {
    "requests": [
        {
            "updateSettings": {
                "settings": {"quizSettings": {"isQuiz": True}},
                "updateMask": "quizSettings.isQuiz",
            }
        }
    ]
}

# Creates the initial form
result = form_service.forms().create(body=NEW_FORM).execute()

question_setting = (
    form_service.forms()
    .batchUpdate(formId=result["formId"], body=UPDATE)
    .execute()
)

# Prints the result to show the question has been added
get_result = form_service.forms().get(formId=result["formId"]).execute()
print(get_result)


{'formId': '1S5nfp8_LdGvTIej4ELs5jAaHv36X-gXSqBOu4Lfzppk', 'info': {'title': 'Guía de estudio para SAFe', 'documentTitle': 'Untitled form'}, 'settings': {'quizSettings': {'isQuiz': True}}, 'revisionId': '00000004', 'responderUri': 'https://docs.google.com/forms/d/e/1FAIpQLSevvYCXAtScsgQfMDTIzGPlojkCAgb-v4AQkIujp3wVMFHrng/viewform'}


In [92]:
def set_question(item, question):
    item["requests"][0]["createItem"]["item"]["title"] = question

def set_correct_answer(item, answer):
    item["requests"][0]["createItem"]["item"]["questionItem"]["question"]["grading"]["correctAnswers"]["answers"][0]["value"] = answer

def set_options_for_question(item, options):
    list_of_options = []
    for option in options:
        list_of_options.append({'value': option})
    item["requests"][0]["createItem"]["item"]["questionItem"]["question"]["choiceQuestion"]["options"] = list_of_options

In [93]:
answers = get_correct_answers()
questions_and_options = get_questions_and_options()
questions = list(questions_and_options.keys())
options = list(questions_and_options.values())

In [94]:
def create_question(item, question, opt, answer):
    set_question(item, question)
    set_options_for_question(item, opt)
    set_correct_answer(item, answer)

def create_questions(n, questions, opt, answers):
    FORMAT_QUESTION = {
        "requests": [
            {
                "createItem": {
                    "item": {
                        "title": (
                            "In what year did the United States land a mission on"
                            " the moon?"
                        ),
                        "questionItem": {
                            "question": {
                                "required": True,
                                "grading": {
                                    "pointValue": 1,
                                    "correctAnswers": {
                                        "answers": [
                                            {
                                                "value": "1967"
                                            }
                                        ]
                                    }
                                },
                                "choiceQuestion": {
                                    "type": "RADIO",
                                    "options": [
                                        {"value": "1965"},
                                        {"value": "1967"},
                                        {"value": "1969"},
                                        {"value": "1971"},
                                    ],
                                    "shuffle": False,
                                },
                            }
                        },
                    },
                    "location": {"index": 0},
                }
            }
        ]
    }
    for i in range(n):
        set_question(FORMAT_QUESTION, questions[i])
        set_options_for_question(FORMAT_QUESTION, opt[i])
        set_correct_answer(FORMAT_QUESTION, answers[i])
        question_setting = (
            form_service.forms()
            .batchUpdate(formId=result["formId"], body=FORMAT_QUESTION)
            .execute()
        )

In [95]:
# Create forms
n = len(questions)
create_questions(n, questions, options, answers)
get_result = form_service.forms().get(formId=result["formId"]).execute()
print(get_result)

{'formId': '1S5nfp8_LdGvTIej4ELs5jAaHv36X-gXSqBOu4Lfzppk', 'info': {'title': 'Guía de estudio para SAFe', 'documentTitle': 'Untitled form'}, 'settings': {'quizSettings': {'isQuiz': True}}, 'revisionId': '00000031', 'responderUri': 'https://docs.google.com/forms/d/e/1FAIpQLSevvYCXAtScsgQfMDTIzGPlojkCAgb-v4AQkIujp3wVMFHrng/viewform', 'items': [{'itemId': '6a3fa968', 'title': 'What is one anti-pattern of the Inspect and Adapt?', 'questionItem': {'question': {'questionId': '20fcc249', 'required': True, 'grading': {'pointValue': 1, 'correctAnswers': {'answers': [{'value': 'No actionable improvement Features are created'}]}}, 'choiceQuestion': {'type': 'RADIO', 'options': [{'value': 'Not enough team members attend the PI System demo'}, {'value': 'No actionable improvement Features are created'}, {'value': 'Too many ideas enter the problem-solving workshop'}, {'value': 'Only one problem is identified by each team in the retrospective'}]}}}}, {'itemId': '113578e7', 'title': 'What is one way un